In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import os.path

In [2]:
PATH = "./data/"
PATH_MY_DGE = "./data/DESeq_results/"

LRT = "LRT/res_LRT.xlsx"

MEAN_DAL = "mean/DAL/res_mean.xlsx"
MEAN_V2 = "mean/V2/res_mean.xlsx"
MEAN_V3 = "mean/V3/res_mean.xlsx"
MEAN_AB_KCs = "mean/AB_KCs/res_mean.xlsx"
MEAN_G_KCs = "mean/G_KCs/res_mean.xlsx"
MEAN_R27 = "mean/R27/res_mean.xlsx"
MEAN_G386 = "mean/G386/res_mean.xlsx"

PAIRWISE_DAL = "pairwise/res_DAL_"
PAIRWISE_V2 = "pairwise/res_V2_"
PAIRWISE_V3 = "pairwise/res_V3_"
PAIRWISE_AB_KCs = "pairwise/res_AB_KCs_"
PAIRWISE_G_KCs = "pairwise/res_G_KCs_"
PAIRWISE_R27 = "pairwise/res_R27_"
PAIRWISE_G386 = "pairwise/res_G386_"

DGE_PAPER = "./data/NIHMS780544-supplement-5.xlsx"

COGNITION_GENE_NAMES = "FlyBase_Fields_download.txt"

In [3]:
NB_COGNITION_GENES = 137
NB_GENES = 15682

In [4]:
# Names of genes related to cognition from FlyBase (names converted on flybase.org directly)
cognition_genes = pd.read_csv(PATH + COGNITION_GENE_NAMES, delimiter = '\t')
#  Store these gene names in a list
cognition_gene_names = list(cognition_genes['SYMBOL'])

In [5]:
# Counts data
genes_LRT = pd.read_excel(PATH_MY_DGE + LRT, index_col=0)
genes_LRT.dropna(axis=1, inplace=True)
genes_LRT.head()

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
gene,,,,,,
40512,15.273559,19.183609,2.879341,17.970839,6.305485e-03,1.751580e-02
128up,166.372302,0.759255,0.678395,22.132593,1.145669e-03,3.903124e-03
14-3-3epsilon,11584.976852,0.059121,0.172566,20.264403,2.484639e-03,7.696410e-03
14-3-3zeta,58800.016238,-1.867902,0.165862,383.209783,1.135976e-79,2.372373e-76
2mit,5687.472251,-2.184018,0.308143,134.197652,1.677618e-26,7.268750e-25


In [6]:
## TODO: plot the expression levels of the genes that are significant for each neuron type (like boxplots) --> how to normalize/compare the expression levels??
#ax = sns.boxplot(x="day", y="total_bill", data=tips)

In [7]:
def compute_contingency_table(results_table):
    nb_sig = results_table.shape[0]

    results_table_cognition = results_table[results_table['gene'].isin(cognition_gene_names)].copy()
    nb_cognition_sig = results_table_cognition.shape[0]
    nb_cognition_non_sig = NB_COGNITION_GENES - nb_cognition_sig

    results_table_non_cognition = results_table[~results_table['gene'].isin(cognition_gene_names)].copy()
    nb_non_cognition_sig = nb_sig - nb_cognition_sig
    nb_non_cognition_non_sig = NB_GENES - NB_COGNITION_GENES - nb_non_cognition_sig

    contingency_table = pd.DataFrame()
    contingency_table['names'] = ['cognition', 'non-cognition']
    contingency_table['significant'] = [nb_cognition_sig, nb_non_cognition_sig]
    contingency_table['non-significant'] = [nb_cognition_non_sig, nb_non_cognition_non_sig]
    contingency_table.set_index('names', inplace=True)

    return contingency_table

In [8]:
def compute_odds_ratio(contingency_table):
    odd_numerator = contingency_table.iloc[0,0] / contingency_table.iloc[1,0]
    odd_denominator = contingency_table.iloc[0,1] / contingency_table.iloc[1,1]
    odds_ratio = odd_numerator / odd_denominator
    return odds_ratio

In [9]:
def prop_top_list(results_table):
    results_table.sort_values(by=['pvalue'], inplace=True)
    print(results_table) # compute the proportion of genes related to cognition in the top 20 (maybe a bit redundant with the odds ratio)? output the top 20 genes related to cognition?

In [11]:
prop_top_list(genes_mean_DAL)

        gene      baseMean  log2FoldChange     lfcSE       stat        pvalue  \
1734  Pka-C1  13758.814627        2.351329  0.153025  15.365620  2.783615e-53   
1591     mub  24535.829534        2.692266  0.175931  15.302967  7.304876e-53   
1287  Eip93F   9992.995895        2.730567  0.200906  13.591285  4.510912e-42   
52      Appl  24678.701098        1.798225  0.134098  13.409777  5.299777e-41   
2157   VGlut  25766.143689       -2.754349  0.211979 -12.993507  1.331823e-38   
...      ...           ...             ...       ...        ...           ...   
737   CG3857    416.365584        0.810179  0.315113   2.571074  1.013836e-02   
414   CG1602     29.854997        1.683262  0.654853   2.570441  1.015692e-02   
417   CG1631      4.840215        2.026627  0.788568   2.570008  1.016963e-02   
1271     ear    164.206692        1.243725  0.484137   2.568952  1.020065e-02   
2084    TMS1   1999.036407       -0.685197  0.266867  -2.567558  1.024176e-02   

              padj  
1734  

In [10]:
genes_mean_DAL = pd.read_excel(PATH_MY_DGE + MEAN_DAL)
genes_mean_DAL.head()

,gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,14-3-3zeta,58800.016238,1.266048,0.108140,11.707467,1.167143e-31,1.135312e-28
1,26-29-p,430.720199,-0.811166,0.311027,-2.608029,9.106532e-03,4.540536e-02
2,2mit,5687.472251,1.703953,0.200415,8.502102,1.861884e-17,3.495116e-15
3,4EHP,680.858997,1.105202,0.298069,3.707870,2.090103e-04,1.980877e-03
4,5-HT1A,1726.547143,1.100984,0.271164,4.060212,4.902812e-05,5.557212e-04


In [35]:
contingency_table_DAL = compute_contingency_table(genes_mean_DAL)
print(contingency_table_DAL)
odds_ratio_DAL = compute_odds_ratio(contingency_table_DAL)
print("Odds ration for DAL vs mean:  %.2f" % odds_ratio_DAL)

               significant  non-significant
names                                      
cognition               56               81
non-cognition         2141            13404
Odds ration for DAL vs mean:  4.33


In [64]:
genes_mean_V2 = pd.read_excel(PATH_MY_DGE + MEAN_V2)
genes_mean_V2.head()

,gene,baseMean,log2FoldChange,lfcSE,stat,pvalue
0,14-3-3zeta,58800.016238,-0.434886,0.108152,-4.021061,5.793665e-05
1,2mit,5687.472251,-0.942410,0.200533,-4.699520,2.607731e-06
2,5-HT1A,1726.547143,-1.653347,0.271613,-6.087146,1.149410e-09
3,5-HT1B,633.144348,-1.028352,0.315094,-3.263635,1.099927e-03
4,7B2,4951.678226,1.010074,0.142764,7.075143,1.492952e-12


In [37]:
contingency_table_V2 = compute_contingency_table(genes_mean_V2)
print(contingency_table_V2)
odds_ratio_V2 = compute_odds_ratio(contingency_table_V2)
print("Odds ration for V2 vs mean:  %.2f" % odds_ratio_V2)

               significant  non-significant
names                                      
cognition               29              108
non-cognition         1034            14511
Odds ration for V2 vs mean:  3.77


In [38]:
genes_mean_V3 = pd.read_excel(PATH_MY_DGE + MEAN_V3)#, index_col=0)
genes_mean_V3.head()

,gene,baseMean,log2FoldChange,lfcSE,stat,pvalue
0,140up,103.708171,-1.829006,0.618203,-2.958586,3.090537e-03
1,14-3-3epsilon,11584.976852,-0.318880,0.123701,-2.577816,9.942686e-03
2,14-3-3zeta,58800.016238,-0.822359,0.118805,-6.921909,4.455983e-12
3,4EHP,680.858997,-0.799613,0.326712,-2.447456,1.438685e-02
4,5-HT1A,1726.547143,1.018431,0.296849,3.430808,6.017868e-04


In [39]:
contingency_table_V3 = compute_contingency_table(genes_mean_V3)
print(contingency_table_V3)
odds_ratio_V3 = compute_odds_ratio(contingency_table_V3)
print("Odds ration for V3 vs mean:  %.2f" % odds_ratio_V3)

               significant  non-significant
names                                      
cognition               60               77
non-cognition         2796            12749
Odds ration for V3 vs mean:  3.55


In [40]:
genes_mean_AB_KCs = pd.read_excel(PATH_MY_DGE + MEAN_AB_KCs)
genes_mean_AB_KCs.head()

,gene,baseMean,log2FoldChange,lfcSE,stat,pvalue
0,40421,164.820080,-1.248336,0.419357,-2.976787,2.912862e-03
1,40512,15.273559,4.001998,0.978795,4.088700,4.337969e-05
2,128up,166.372302,1.500860,0.420228,3.571536,3.548939e-04
3,14-3-3zeta,58800.016238,-0.740799,0.108170,-6.848494,7.463147e-12
4,2mit,5687.472251,-0.717182,0.200585,-3.575460,3.496127e-04


In [19]:
contingency_table_AB_KCs = compute_contingency_table(genes_mean_AB_KCs)
print(contingency_table_AB_KCs)
odds_ratio_AB_KCs = compute_odds_ratio(contingency_table_AB_KCs)
print("Odds ration for AB_KCs vs mean:  %.2f" % odds_ratio_AB_KCs)

               significant  non-significant
names                                      
cognition               59               78
non-cognition         2938            12607
Odds ration for AB_KCs vs mean:  3.25


In [20]:
genes_mean_G_KCs = pd.read_excel(PATH_MY_DGE + MEAN_G_KCs)
genes_mean_G_KCs.head()

,gene,baseMean,log2FoldChange,lfcSE,stat,pvalue
0,14-3-3zeta,58800.016238,1.212506,0.100407,12.075936,1.415449e-33
1,2mit,5687.472251,0.880631,0.186286,4.727309,2.275151e-06
2,4EHP,680.858997,0.775678,0.277527,2.794965,5.190541e-03
3,5-HT1B,633.144348,1.120156,0.292865,3.824823,1.308661e-04
4,7SLRNA:CR32864,110.742163,-1.429284,0.304161,-4.699099,2.613116e-06


In [21]:
contingency_table_G_KCs = compute_contingency_table(genes_mean_G_KCs)
print(contingency_table_G_KCs)
odds_ratio_G_KCs = compute_odds_ratio(contingency_table_G_KCs)
print("Odds ration for G_KCs vs mean:  %.2f" % odds_ratio_G_KCs)

               significant  non-significant
names                                      
cognition               53               84
non-cognition         2174            13371
Odds ration for G_KCs vs mean:  3.88


In [41]:
genes_mean_R27 = pd.read_excel(PATH_MY_DGE + MEAN_R27)
genes_mean_R27.head()

,gene,baseMean,log2FoldChange,lfcSE,stat,pvalue
0,7SLRNA:CR32864,110.742163,-1.336932,0.325908,-4.102175,4.092844e-05
1,7SLRNA:CR42652,112.443580,-1.267791,0.323672,-3.916903,8.969369e-05
2,a10,199.626819,-4.520713,0.831277,-5.438275,5.379880e-08
3,abba,40.742169,-2.753504,0.874904,-3.147209,1.648370e-03
4,Ace,1058.383610,0.931375,0.297593,3.129696,1.749873e-03


In [42]:
contingency_table_R27 = compute_contingency_table(genes_mean_R27)
print(contingency_table_R27)
odds_ratio_R27 = compute_odds_ratio(contingency_table_R27)
print("Odds ration for R27 vs mean:  %.2f" % odds_ratio_R27)

               significant  non-significant
names                                      
cognition               10              127
non-cognition          793            14752
Odds ration for R27 vs mean:  1.46


In [43]:
genes_mean_G386 = pd.read_excel(PATH_MY_DGE + MEAN_G386)
genes_mean_G386.head()

,gene,baseMean,log2FoldChange,lfcSE,stat,pvalue
0,14-3-3zeta,58800.016238,-0.593148,0.108155,-5.484214,4.153111e-08
1,5-HT1A,1726.547143,-1.106958,0.271466,-4.077703,4.548277e-05
2,a10,199.626819,4.703805,0.775988,6.061701,1.346894e-09
3,a5,10.039765,3.248448,0.977812,3.322160,8.932334e-04
4,Ac78C,289.484002,-1.429288,0.432572,-3.304164,9.526025e-04


In [44]:
contingency_table_G386 = compute_contingency_table(genes_mean_G386)
print(contingency_table_G386)
odds_ratio_G386 = compute_odds_ratio(contingency_table_G386)
print("Odds ration for G386 vs mean:  %.2f" % odds_ratio_G386)

               significant  non-significant
names                                      
cognition               19              118
non-cognition          544            15001
Odds ration for G386 vs mean:  4.44


In [45]:
genes_DAL_V2 = pd.read_excel(PATH_MY_DGE + PAIRWISE_DAL + "V2.xlsx", index_col=0)
genes_DAL_V3 = pd.read_excel(PATH_MY_DGE + PAIRWISE_DAL + "V3.xlsx", index_col=0)
genes_DAL_AB_KCs = pd.read_excel(PATH_MY_DGE + PAIRWISE_DAL + "AB_KCs.xlsx", index_col=0)
genes_DAL_G_KCs = pd.read_excel(PATH_MY_DGE + PAIRWISE_DAL + "G_KCs.xlsx", index_col=0)
genes_DAL_R27 = pd.read_excel(PATH_MY_DGE + PAIRWISE_DAL + "R27.xlsx", index_col=0)
genes_DAL_G386 = pd.read_excel(PATH_MY_DGE + PAIRWISE_DAL + "G386.xlsx", index_col=0)

In [21]:
genes_V2_DAL = pd.read_excel(PATH_MY_DGE + PAIRWISE_V2 + "DAL.xlsx", index_col=0)
genes_V2_V3 = pd.read_excel(PATH_MY_DGE + PAIRWISE_V2 + "V3.xlsx", index_col=0)
genes_V2_AB_KCs = pd.read_excel(PATH_MY_DGE + PAIRWISE_V2 + "AB_KCs.xlsx", index_col=0)
genes_V2_G_KCs = pd.read_excel(PATH_MY_DGE + PAIRWISE_V2 + "G_KCs.xlsx", index_col=0)
genes_V2_R27 = pd.read_excel(PATH_MY_DGE + PAIRWISE_V2 + "R27.xlsx", index_col=0)
genes_V2_G386 = pd.read_excel(PATH_MY_DGE + PAIRWISE_V2 + "G386.xlsx", index_col=0)

In [22]:
genes_V3_DAL = pd.read_excel(PATH_MY_DGE + PAIRWISE_V3 + "DAL.xlsx", index_col=0)
genes_V3_V2 = pd.read_excel(PATH_MY_DGE + PAIRWISE_V3 + "V2.xlsx", index_col=0)
genes_V3_AB_KCs = pd.read_excel(PATH_MY_DGE + PAIRWISE_V3 + "AB_KCs.xlsx", index_col=0)
genes_V3_G_KCs = pd.read_excel(PATH_MY_DGE + PAIRWISE_V3 + "G_KCs.xlsx", index_col=0)
genes_V3_R27 = pd.read_excel(PATH_MY_DGE + PAIRWISE_V3 + "R27.xlsx", index_col=0)
genes_V3_G386 = pd.read_excel(PATH_MY_DGE + PAIRWISE_V3 + "G386.xlsx", index_col=0)

In [23]:
genes_AB_KCs_DAL = pd.read_excel(PATH_MY_DGE + PAIRWISE_AB_KCs + "DAL.xlsx", index_col=0)
genes_AB_KCs_V2 = pd.read_excel(PATH_MY_DGE + PAIRWISE_AB_KCs + "V2.xlsx", index_col=0)
genes_AB_KCs_V3 = pd.read_excel(PATH_MY_DGE + PAIRWISE_AB_KCs + "V3.xlsx", index_col=0)
genes_AB_KCs_G_KCs = pd.read_excel(PATH_MY_DGE + PAIRWISE_AB_KCs + "G_KCs.xlsx", index_col=0)
genes_AB_KCs_R27 = pd.read_excel(PATH_MY_DGE + PAIRWISE_AB_KCs + "R27.xlsx", index_col=0)
genes_AB_KCs_G386 = pd.read_excel(PATH_MY_DGE + PAIRWISE_AB_KCs + "G386.xlsx", index_col=0)

In [24]:
genes_G_KCs_DAL = pd.read_excel(PATH_MY_DGE + PAIRWISE_G_KCs + "DAL.xlsx", index_col=0)
genes_G_KCs_V2 = pd.read_excel(PATH_MY_DGE + PAIRWISE_G_KCs + "V2.xlsx", index_col=0)
genes_G_KCs_V3 = pd.read_excel(PATH_MY_DGE + PAIRWISE_G_KCs + "V3.xlsx", index_col=0)
genes_G_KCs_AB_KCs = pd.read_excel(PATH_MY_DGE + PAIRWISE_G_KCs + "AB_KCs.xlsx", index_col=0)
genes_G_KCs_R27 = pd.read_excel(PATH_MY_DGE + PAIRWISE_G_KCs + "R27.xlsx", index_col=0)
genes_G_KCs_G386 = pd.read_excel(PATH_MY_DGE + PAIRWISE_G_KCs + "G386.xlsx", index_col=0)

In [25]:
genes_R27_DAL = pd.read_excel(PATH_MY_DGE + PAIRWISE_R27 + "DAL.xlsx", index_col=0)
genes_R27_V2 = pd.read_excel(PATH_MY_DGE + PAIRWISE_R27 + "V2.xlsx", index_col=0)
genes_R27_V3 = pd.read_excel(PATH_MY_DGE + PAIRWISE_R27 + "V3.xlsx", index_col=0)
genes_R27_AB_KCs = pd.read_excel(PATH_MY_DGE + PAIRWISE_R27 + "AB_KCs.xlsx", index_col=0)
genes_R27_G_KCs = pd.read_excel(PATH_MY_DGE + PAIRWISE_R27 + "G_KCs.xlsx", index_col=0)
genes_R27_G386 = pd.read_excel(PATH_MY_DGE + PAIRWISE_R27 + "G386.xlsx", index_col=0)

In [26]:
genes_G386_DAL = pd.read_excel(PATH_MY_DGE + PAIRWISE_G386 + "DAL.xlsx", index_col=0)
genes_G386_V2 = pd.read_excel(PATH_MY_DGE + PAIRWISE_G386 + "V2.xlsx", index_col=0)
genes_G386_V3 = pd.read_excel(PATH_MY_DGE + PAIRWISE_G386 + "V3.xlsx", index_col=0)
genes_G386_AB_KCs = pd.read_excel(PATH_MY_DGE + PAIRWISE_G386 + "AB_KCs.xlsx", index_col=0)
genes_G386_G_KCs = pd.read_excel(PATH_MY_DGE + PAIRWISE_G386 + "G_KCs.xlsx", index_col=0)
genes_G386_R27 = pd.read_excel(PATH_MY_DGE + PAIRWISE_G386 + "R27.xlsx", index_col=0)

In [76]:
DAL = pd.read_excel(DGE_PAPER, sheet_name="Supplemental Table 6", usecols="A:H", skiprows=7)
DAL.dropna(axis=0, inplace=True)
DAL.head

,Genes,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,qvalue
0,CG9795,391.861,4.59727,0.745981,6.16271,7.15e-10,2.41e-06,2.22644e-06
1,Iris,613.726,4.31574,0.698002,6.18299,6.29e-10,2.41e-06,2.22644e-06
2,CG14872,306.428,6.62435,1.19361,5.54983,2.86e-08,5.46e-05,5.04452e-05
3,EndoGI,324.007,6.94372,1.25609,5.52806,3.24e-08,5.46e-05,5.04452e-05
4,CG7488,166.63,7.87991,1.49139,5.2836,1.27e-07,0.000171002,0.000158186
...,...,...,...,...,...,...,...,...
7330,,,,,,,,
7331,,,,,,,,
7332,,,,,,,,
7333,,,,,,,,
